In [3]:
from openai import Client
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")
client = Client()

In [4]:
assistant = client.beta.assistants.create(
    name="AWS Solutions Architect Assistant",
    instructions="You are a professional AWS solutions architect. You are asked to design a solution for the customer. Try to find out the requirements first and then design a solution for the customer.",
    model="gpt-4-1106-preview",
    tools=[{
        "type": "function",
        "function": {
        "name": "getBestPracticeExample",
        "description": "Gets sample best practice example for a given search term",
        "parameters": {
        "type": "object",
        "properties": {
            "query": {"type": "string"}
        },
        "required": ["query"]
      }
    }
  }]
)

In [30]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please help me with best practice samples for EC2 with RDS!"
)

In [31]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)
run

Run(id='run_h8IEEVkeAFPpSkk1t2a1lS6Y', assistant_id='asst_6zdQjDQHaxQNYbRFkjuQD24d', cancelled_at=None, completed_at=None, created_at=1701894504, expires_at=1701895104, failed_at=None, file_ids=[], instructions='You are a professional AWS solutions architect. You are asked to design a solution for the customer. Try to find out the requirements first and then design a solution for the customer.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_CgrNK5Xvh6ZDRWhfqRXOxFEj', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='getBestPracticeExample', parameters={'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query']}, description='Gets sample best practice example for a given search term'), type='function')])

In [65]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
run.status

'completed'

In [66]:
print(run)

Run(id='run_h8IEEVkeAFPpSkk1t2a1lS6Y', assistant_id='asst_6zdQjDQHaxQNYbRFkjuQD24d', cancelled_at=None, completed_at=1701894571, created_at=1701894504, expires_at=None, failed_at=None, file_ids=[], instructions='You are a professional AWS solutions architect. You are asked to design a solution for the customer. Try to find out the requirements first and then design a solution for the customer.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1701894525, status='completed', thread_id='thread_CgrNK5Xvh6ZDRWhfqRXOxFEj', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='getBestPracticeExample', parameters={'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query']}, description='Gets sample best practice example for a given search term'), type='function')])


In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
db = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db")

In [37]:
print(len(run.required_action.submit_tool_outputs.tool_calls))

2


In [38]:
import json


if(run.status == "requires_action"):
  if(run.required_action):
    tool_outputs = []
    x = 0
    # iterate over tool calls and submit the required action
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
      print(tool_call.id)
      match tool_call.function.name:
        case "getBestPracticeExample":
          # parse arguments from tool_call.function.arguments, which is a string
          query = json.loads(tool_call.function.arguments)["query"]
          x = x + 1

          docs = db.similarity_search(query)
          content = ""
          if(len(docs) > 0):
            content = docs[0].page_content
          tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": content
          })
          
    print(tool_outputs)
    print(x)
    run = client.beta.threads.runs.submit_tool_outputs(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
        
    


call_qN7iJysHr2aHgHq23Lfr7F9G
call_RXuqnhXKbf08iPeHJjJfXntb
[{'tool_call_id': 'call_qN7iJysHr2aHgHq23Lfr7F9G', 'output': '# Configure the AWS provider\nprovider "aws" {\n  region = "eu-west-1"\n}\n\n# Create a DB instance\nresource "aws_db_instance" "example" {\n  engine              = "mysql"\n  allocated_storage   = 10\n  instance_class      = "db.t2.micro"\n  name                = "example_database_prod"\n  username            = "admin"\n  password            = "${var.db_password}"\n  skip_final_snapshot = true\n}\n'}, {'tool_call_id': 'call_RXuqnhXKbf08iPeHJjJfXntb', 'output': '# Configure the AWS provider\nprovider "aws" {\n  region = "eu-west-1"\n}\n\n# Create a DB instance\nresource "aws_db_instance" "example" {\n  engine              = "mysql"\n  allocated_storage   = 10\n  instance_class      = "db.t2.micro"\n  name                = "example_database_stage"\n  username            = "admin"\n  password            = "${var.db_password}"\n  skip_final_snapshot = true\n}\n'}]
2


In [70]:
from pprint import pprint


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_Cz1Jgnp2t8RZhA7JjLmYtQrw', assistant_id='asst_6zdQjDQHaxQNYbRFkjuQD24d', content=[MessageContentText(text=Text(annotations=[], value='Here are some best practice examples for using EC2 and RDS:\n\n### Best Practice Example for EC2:\nPlease note that EC2 best practices can depend on specific use cases. However, there are no sample Terraform configurations provided for EC2 best practices. Nevertheless, I can give you a few general tips:\n- Select the right instance size and type based on your workload.\n- Use Amazon Machine Images (AMIs) to launch EC2 instances with preconfigured settings.\n- Implement security groups and network access control lists (ACLs) to control traffic to instances.\n- Enable monitoring and logging using Amazon CloudWatch for instance performance and health checks.\n- Regularly apply patches and updates to your EC2 instances to ensure security.\n- Leverage Auto Scaling to adjust capacity according to condit

In [7]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain.agents import AgentExecutor


In [15]:
tools = [DuckDuckGoSearchRun()]
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal assistant. You can search the internet.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's 10 - 4 raised to the 2.7"})





RateLimitException: _get_url() https://links.duckduckgo.com/d.js

In [14]:
run

[OpenAIAssistantAction(tool='duckduckgo_search', tool_input={'query': 'weather in San Francisco today'}, log='', tool_call_id='call_Ae2l6GYnHOCwkv9O77OBtxe5', run_id='run_KOvMP2YX3ldE0jQR6Br20wIM', thread_id='thread_tqGwbIc5Mj9hq791rtyZ5iSA')]